In [222]:
import random

class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[: len(negative)]
        self.reviews = positive_shrunk + negative
        random.shuffle(self.reviews)

In [223]:
import json

file_name = 'books_small_10000.json'
reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [224]:
len(reviews)

10000

In [225]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

train_container.evenly_distribute()
test_container.evenly_distribute()

In [226]:
len(training)

6700

In [227]:
print(training[0].sentiment)

POSITIVE


In [228]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

In [229]:
train_x[1]

"The first book I understood all the drama and everything that led up to the second book but she you get to the second I understand the hard feeling between the characters in the beginning but it just keeps going and it seems to be unnecessary it never calms down at any point I liked most of the book in a whole but I feel like the continued drama was crazy and the way it ended I hated it,it just doesn't seem to fit"

In [230]:
train_y[1]

'NEGATIVE'

In [231]:
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [232]:
test_x[0]

"I really hate writing bad reviews almost as much as I hate reading unbelievable characters in a story.  The writing wasn't bad but the content and how the characters were presented disturbed me (and not because of the porn-like scenes).  Conflict is good to keep a story going but out of character behavior just for shock value is a turn off."

In [233]:
test_y[0]

'NEGATIVE'

In [234]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [235]:
train_x_vectors

<872x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 53552 stored elements in Compressed Sparse Row format>

In [236]:
test_x_vectors

<416x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 24256 stored elements in Compressed Sparse Row format>

In [237]:
test_x_vectors

<416x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 24256 stored elements in Compressed Sparse Row format>

In [238]:
test_x_vectors[0]

<1x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [239]:
from sklearn import svm

In [240]:
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [241]:
test_x[0]

"I really hate writing bad reviews almost as much as I hate reading unbelievable characters in a story.  The writing wasn't bad but the content and how the characters were presented disturbed me (and not because of the porn-like scenes).  Conflict is good to keep a story going but out of character behavior just for shock value is a turn off."

In [242]:
test_y[0]

'NEGATIVE'

In [243]:
print(clf_svm.predict(test_x_vectors[0]))

['NEGATIVE']


In [244]:
user_input = "Overall an average book i would say. Did not enjoy much"

In [245]:
user_input_vector = vectorizer.transform([user_input])

In [246]:
print(user_input_vector)

  (0, 8813)	0.23722381445255136
  (0, 6867)	0.32723758779618994
  (0, 5601)	0.4448494139840391
  (0, 5404)	0.16369842091987755
  (0, 5220)	0.25248161051378204
  (0, 2679)	0.3333766774556179
  (0, 2221)	0.26934732589275684
  (0, 982)	0.13074200317914544
  (0, 659)	0.5483483256097684
  (0, 406)	0.21618958480328404


In [247]:
print(clf_svm.predict(user_input_vector))

['POSITIVE']


In [248]:
print(clf_svm.predict(test_x_vectors))

['NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE'
 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE'
 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE'
 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE'
 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE'
 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE'
 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE'
 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGAT

In [249]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier()

In [250]:
print(clf_dec.predict(user_input_vector[0]))

['NEGATIVE']


In [251]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.todense(), train_y)

GaussianNB()

In [252]:
print(clf_gnb.predict(user_input_vector.todense()))

['NEGATIVE']


In [253]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

LogisticRegression()

In [254]:
print(clf_log.predict(test_x_vectors[2]))

['POSITIVE']


In [255]:
# Mean Accuracy


print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.todense(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.7716346153846154
0.6418269230769231
0.6394230769230769
0.7740384615384616


In [257]:
# F1 Scores

from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors.todense()), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.76658477 0.77647059]
[0.64608076 0.63746959]
[0.64454976 0.63414634]
[0.77294686 0.77511962]


In [262]:
test_y.count(Sentiment.POSITIVE)

208

In [263]:
test_set = ['good read', 'bad book do not buy', 'horrible waste of time', 'Overall a mild book i would say. Did not enjoy much']
new_test = vectorizer.transform(test_set)

In [264]:
print(clf_log.predict(new_test))

['POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE']


In [265]:
from sklearn.model_selection import GridSearchCV

parameters = {'solver': ('newton-cg', 'lbfgs','liblinear','sag','saga'), 'C': (1, 2, 3, 4, 8), 'max_iter': (400, 800,1000,1600)}
log_reg = LogisticRegression()

clf = GridSearchCV(log_reg, parameters, cv=5)

clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': (1, 2, 3, 4, 8),
                         'max_iter': (400, 800, 1000, 1600),
                         'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga')})

In [266]:
print(clf.score(test_x_vectors, test_y))

0.7836538461538461


In [267]:
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [268]:
with open('sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [269]:
print(test_y[3])

print(loaded_clf.predict(test_x_vectors[3]))

POSITIVE
['POSITIVE']
